In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('temp.csv', sep=',')

/var/folders/53/kjtpb3yd50l_0wd__m7cm_7m0000gn/T/ipykernel_2109/3746114537.py:1: DtypeWarning: Columns (19,24,25,27,29,30,31,32,33,34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('temp.csv', sep=',')


In [3]:
df.head(1)

,Unnamed: 0,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/07/2017,Vente,"166100,00",843.0,NaN,RUE,0100,DE LA CHARPINE,1000.0,SAINT-DENIS-LES-BOURG,1,344,NaN,AK,534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,NaN,73.0,4.0,S,NaN,391.0


### DATA EXPLO

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18036812 entries, 0 to 18036811
Data columns (total 44 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Unnamed: 0                  int64  
 1   Code service sages          float64
 2   Reference document          float64
 3   1 Articles CGI              float64
 4   2 Articles CGI              float64
 5   3 Articles CGI              float64
 6   4 Articles CGI              float64
 7   5 Articles CGI              float64
 8   No disposition              int64  
 9   Date mutation               object 
 10  Nature mutation             object 
 11  Valeur fonciere             object 
 12  No voie                     float64
 13  B/T/Q                       object 
 14  Type de voie                object 
 15  Code voie                   object 
 16  Voie                        object 
 17  Code postal                 float64
 18  Commune                     object 
 19  Code departement   

In [5]:
df['Nature culture'].unique()

array(['S', nan, 'AB', 'AG', 'T', 'P', 'BS', 'J', 'VI', 'CA', 'L', 'BT',
       'BP', 'E', 'VE', 'BR', 'BM', 'PA', 'BF', 'CH', 'B', 'PP', 'PC',
       'BO', 'TP', 'LB', 'PH', 'PE'], dtype=object)

In [6]:
df['Type local'].unique()

array(['Maison', 'Dépendance', 'Appartement', nan,
       'Local industriel. commercial ou assimilé'], dtype=object)

In [7]:
df['Nature mutation'].unique()

array(['Vente', 'Vente terrain à bâtir', 'Echange',
       "Vente en l'état futur d'achèvement", 'Adjudication',
       'Expropriation'], dtype=object)

In [8]:
df['Code departement'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 22, 23, 24, 25, 26, 27, 28, 29, '29', '2A', '2B', '30', 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
       49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 971, 972, 973, 974, 75],
      dtype=object)

### PIPE

In [9]:
df['Date mutation'] = pd.to_datetime(df['Date mutation'], dayfirst=True)
df['annee'] = df['Date mutation'].dt.year

In [10]:
#Adresse:
my_address_variable = ['No voie', 'B/T/Q', 'Voie']
df = df[~df['Code postal'].isna()]
df = df[~df['Commune'].isna()]
df = df[~df['Voie'].isna()]

df['Code département'] = df['Code departement'].astype('str')
df['Code postal'] = df['Code postal'].astype('str').replace('.0', '')
df['No voie'] = df['No voie'].astype('str')

df['Code postal'] = df['Code postal'].apply(lambda x: '0'+str(x) if len(str(x))<5 else x)
df['Code departement'] = df['Code departement'].apply(lambda x: '0'+str(x) if len(str(x))<2 else x)

df['Adresse'] = df[my_address_variable].astype('str').apply(' '.join, axis=1)

In [11]:
#df['Adresse'].replace('.0', '', inplace=True, regex=True)
df['Adresse'].replace('nan', '', inplace=True, regex=True)
df['Adresse'].replace('  ', ' ', inplace=True, regex=True)

In [12]:
df['Valeur fonciere'] = df['Valeur fonciere'].str.replace(',', '.').astype('float')

In [13]:
df = df[df['Nature mutation'].isin(['Vente', 'Vente terrain à bâtir', "Vente en l'état futur d'achèvement", "Adjudication"])]

In [14]:
df = df[~df['Type local'].isin(['971', '972', '973', '974'])]

In [15]:
#Suppression des 'dépendances' car elles font l'objet d'une mutation indépendantes et apportent peu au futur modèle
df = df[df['Type local'].isin(["Maison", "Appartement", "Local industriel. commercial ou assimilé"])]

In [16]:
my_variables = ['Date mutation', 'Nature mutation', 'Valeur fonciere', 'Code departement', 'Surface reelle bati', 'Type local', 'Nombre pieces principales', 'Surface terrain', 'Nombre de lots', 'Section', 'No plan', 'Adresse', 'Commune', 'Code postal']
#my_index = ['Commune', 'Voie', 'Type de voie', 'No voie', 'B/T/Q', 'Section', 'No plan', 'Date mutation', 'Code departement']
my_index = ['Adresse', 'Commune', 'Code postal', 'Section', 'No plan', 'Date mutation', 'Nature mutation']

my_category = ['Nature mutation', 'Code departement', 'Type local', 'annee']
my_dates = ['Date mutation']
my_values = ['Valeur fonciere', 'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain', 'Nombre de lots']

In [17]:
#filtre selection variables
df = df[my_variables]

In [18]:
df['No plan'] = df['No plan'].astype('str')

In [19]:
#df = df.drop_duplicates(subset=my_index)

In [20]:
df = df.dropna(subset=my_values)


In [21]:
df.columns = df.columns.str.replace(" ", "_")

In [22]:
df.Adresse = df.Adresse.str.replace(".0", "")

/var/folders/53/kjtpb3yd50l_0wd__m7cm_7m0000gn/T/ipykernel_2109/127325063.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.Adresse = df.Adresse.str.replace(".0", "")


In [23]:
df = df[~df['Section'].isna()]

In [24]:
df.to_csv('flat_file_dataframe.csv', index=False)

In [3]:
df= pd.read_csv('flat_file_dataframe.csv')

/var/folders/53/kjtpb3yd50l_0wd__m7cm_7m0000gn/T/ipykernel_8000/1831528561.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('flat_file_dataframe.csv')


In [6]:
test = df.to_dict("index")

In [8]:
test

{0: {'Date_mutation': '2017-07-04',
  'Nature_mutation': 'Vente',
  'Valeur_fonciere': 166100.0,
  'Code_departement': 1,
  'Surface_reelle_bati': 73.0,
  'Type_local': 'Maison',
  'Nombre_pieces_principales': 4.0,
  'Surface_terrain': 391.0,
  'Nombre_de_lots': 0,
  'Section': 'AK',
  'No_plan': 534,
  'Adresse': '843 DE LA CHARPINE',
  'Commune': 'SAINT-DENIS-LES-BOURG',
  'Code_postal': 1000.0},
 1: {'Date_mutation': '2017-07-06',
  'Nature_mutation': 'Vente',
  'Valeur_fonciere': 130000.0,
  'Code_departement': 1,
  'Surface_reelle_bati': 96.0,
  'Type_local': 'Maison',
  'Nombre_pieces_principales': 2.0,
  'Surface_terrain': 471.0,
  'Nombre_de_lots': 0,
  'Section': 'D',
  'No_plan': 4184,
  'Adresse': '5123 HAMEAU DU CREUX EST',
  'Commune': 'REPLONGES',
  'Code_postal': 1750.0},
 2: {'Date_mutation': '2017-07-03',
  'Nature_mutation': 'Vente',
  'Valeur_fonciere': 145650.0,
  'Code_departement': 1,
  'Surface_reelle_bati': 72.0,
  'Type_local': 'Appartement',
  'Nombre_pieces_p